In [1]:
from my_trader import *
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib2
import time
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_driver.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_driver

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [11]:
driver=webdriver.Firefox()
session_id =driver.session_id
executor_url =driver.command_executor._url

In [12]:
driver.get('https://signon.onesource.com/Web/Logon/Signin.aspx')
time.sleep(5)
driver.implicitly_wait(5)
driver.find_element_by_id("username").clear()
driver.find_element_by_id("username").send_keys("hli@calasiancc.org")
driver.find_element_by_id("password").clear()
driver.find_element_by_id("password").send_keys("dbhoovers1")
driver.find_element_by_id("btnSignIn").click()
time.sleep(5)
driver.implicitly_wait(10)
driver.find_element_by_link_text("Lists").click()
time.sleep(5)
driver.implicitly_wait(10)

# list selector
i = 0
list_ele = driver.find_elements_by_class_name("name-row")
get = list_ele[0]
get.find_element_by_tag_name("a").click()

In [13]:
i = 3
company_list = driver.find_elements_by_class_name("clickable")
driver.implicitly_wait(10)
get_company = company_list[i]
driver.execute_script("return arguments[0].scrollIntoView();", get_company)
driver.execute_script("arguments[0].click();", get_company)
#company_list[i].click()

In [87]:
#get basic information
info = {}
data = driver.find_elements_by_class_name("data-container")
for i in range(len(data)):
    
    try:
        label=data[i].find_element_by_class_name("data-label")
    except:
        continue 
        #label=data[i].find_element_by_class_name("data-label wide")
    value=data[i].find_elements_by_class_name("data-value")
    value = value[0].text
    label = label.text
    label = label.encode('ascii','ignore')
    info[label] = value
address = driver.find_element_by_class_name("company-address").text
company_name = driver.find_element_by_class_name("name").text
info["address"] = address
info["company_name"] = company_name
mongodb.db["hoover_data"].insert_one(info)

#get description about the company

data_title = ["description","industry","highlights"]
description=pd.DataFrame()
for i in data_title:
    get_temp = driver.find_element_by_class_name("highlights")
    get_temp = get_temp.text.split("\n")
    #get_temp = get_temp.text.split(":")
for i in range(len(get_temp)):
    get_temp[i] = get_temp[i].encode('ascii','ignore')
    get = get_temp[i].split(":")
    description = description.append([get])
description = description.set_index(0)
description=json.loads(description.to_json()).values()[0]
mongodb.db["hoover_data"].update_one({"company_name":company_name},{"$set":description},upsert=True)

#---------------------------
def render_text_contact(get_temp):
    #flag = False
    result = pd.DataFrame()
    get_temp = get_temp.text.split("\n")
    header=["name","position","location","industry"]
    count_header = 0
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        if ":" in get_temp[i]:
            #flag = True
            get = get_temp[i].split(":")
            result = result.append([get])
            
        else:
            result = result.append([[header[count_header],get_temp[i]]])
            count_header +=1
        
    result = result.reindex()    
    return result
#---------------------------
contacts = driver.find_element_by_id("totalContacts").text
contacts = int(contacts)
if contacts != 0:
    driver.find_element_by_id("company_contacts").click()
    # find contact
contacts = driver.find_element_by_id("totalContacts").text
contacts = int(contacts)
if contacts != 0:
    driver.find_element_by_id("company_contacts").click()
    contacts = driver.find_elements_by_class_name("detail-container")

    contacts = driver.find_elements_by_class_name("search-result-container")
    result = pd.DataFrame()
    for i in range(len(contacts)):

        result = result.append(render_text_contact(contacts[i]))

    # reindexing

    for i in range(len(result)):
        index_name = i/4
        result.iloc[i].name=index_name
    x=range(len(result.index)/5 )
    index_name=np.repeat(x,5)
    result.index = index_name

    # organize contact in to a dataframe
    new_result = pd.DataFrame()
    for i in list(set(result.index)):
        new_result= new_result.append([list(result.loc[i].T.iloc[1].values)])
    new_result.columns=list(result.loc[0].T[0:1].iloc[0])  
time.sleep(3)
driver.implicitly_wait(10)    
driver.back()
new_result = json.loads(new_result.T.to_json()).values()
mongodb.db["hoover_data"].update_one({"company_name":company_name},{"$set":{"contact":new_result}},upsert=True)

In [245]:
#page info

current_page = driver.find_element_by_class_name("current-page")
current_page=current_page.get_attribute('innerHTML')
current_page = int(current_page)

total_page = driver.find_element_by_class_name("total-page")
total_page=total_page.get_attribute('innerHTML')
total_page = int(total_page)


if total_page > current_page:
    next_page = driver.find_element_by_class_name("next-page")
    driver.execute_script("arguments[0].click();", next_page)
else:
    break

In [3]:
mongodb = mongo("hoover")

In [113]:
pd.DataFrame(list(mongodb.db["hoover_data"].find()))

,,Annual Sales:,Company Type:,Corporate Highlights,D-U-N-S Number,D-U-N-S Number:,Employees:,Incorporation Date,Industry:,Key ID Number:,Latitude,Longitude,Minority Owned,Reporting Currency:,Tel:,Tradestyle,_id,address,company_name
0,,41k,Private Independent,None,080983889,08-098-3889,1 (This Site) 1 (All Sites),2017,Nonclassifiable Establishments,278850734,38.589226,-121.405677,True,USD,916-549-8959,Alexander Builds,5aa464e57ed7aa452174aa62,"2020 Hurley Way Ste 225\nSacramento, Californi...",Bryan Alexander


In [98]:
contacts[i]

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="b5c77f41-a284-ec43-bee1-6674bbc8ea2c", element="54e34c9d-45e5-144b-93bc-a4b7d24bd501")>

In [67]:
description=json.loads(description.to_json()).values()

AttributeError: 'list' object has no attribute 'to_json'

In [75]:
description['Incorporation Date']

u' 2017'

In [107]:
new_result = json.loads(new_result.to_json()).values()

AttributeError: 'dict' object has no attribute 'to_json'

In [115]:
new_result

,name,position,location,industry,Main
0,Tim Costa,"President at W B Wright, Inc","Sacramento, California, United States",Food Manufacturing,916-443-2346


In [124]:
new_result=json.loads(new_result.T.to_json()).values()

In [122]:
new_result

,name,position,location,industry,Main
0,Tim Costa,"President at W B Wright, Inc","Sacramento, California, United States",Food Manufacturing,916-443-2346
